# Task 2 - Building the data set

In [12]:
# Imports
import random
import torch
import torch.nn as nn
import torch.optim as optim
import time
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import requests

In [ ]:
# Choose a device
device = "cuda:0"

### Downloading the required images
50 - male faces
50 - female faces
50 - other images

In [13]:
data_request = {"male" : "https://fakeface.rest/face/view?gender=male",
                "female" : "https://fakeface.rest/face/view?gender=female",
                "pokemon" : "https://pokeres.bastionbot.org/images/pokemon/"
                }
for current_set in data_request.items():
    for i in range(0, 50):
        if current_set[0] == "pokemon":
            response = requests.get(current_set[1] + str(random.randint(1, 890)) + ".png")
        else:
            response = requests.get(current_set[1])
        file = open("data/face_set/" + current_set[0] + "/" + str(i + 1) + ".png", "wb")
        file.write(response.content)
        file.close()

In [ ]:
class ImageClassifierDataset(Dataset):
    def __init__(self, image_list, image_classes):
        self.images = []
        self.labels = []

        self.classes = list(set(image_classes))
        self.class_to_label = { c: i for i, c in enumerate(self.classes) }
        self.image_size = 224
        self.transforms = transforms.Compose([
            transforms.Resize(self.image_size),
            transforms.CenterCrop(self.image_size),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        for image, image_class in zip(image_list, image_classes):
            transformed_image = self.transforms(image)
            self.images.append(transformed_image)
            label = self.class_to_label[image_class]
            self.labels.append(label)

    def __getitem__(self, index):
        return self.images[index], self.labels[index]
    def __len__(self):
        return len(self.images)
